# Hyperliquid Data Overview

This notebook provides a complete overview of Hyperliquid's publicly available data infrastructure.

**Goal**: Build a trading engine that deeply understands Hyperliquid through its traders.

---

## Setup

Create `.env` from `.env.example`:
```
AWS_ACCESS_KEY_ID=...
AWS_SECRET_ACCESS_KEY=...
AWS_REGION=us-east-1
```

In [18]:
from collections import Counter
from datetime import datetime
import json

from vigil import (
    download,
    get_s3_client,
    list_files,
    list_prefixes,
    parse_jsonl_lz4,
    parse_msgpack_lz4,
)

s3 = get_s3_client()

print("Ready")

Ready


---

## The Two S3 Buckets

From [Hyperliquid docs](https://hyperliquid.gitbook.io/hyperliquid-docs/historical-data):

| Bucket | Purpose | Data Types |
|--------|---------|------------|
| `hyperliquid-archive` | Market data archives | Market data |
| `hl-mainnet-node-data` | Node-streamed data | Explorer blocks, trades, fills |


In [19]:
print("=== hyperliquid-archive ===")
for p in list_prefixes('hyperliquid-archive'):
    print(f"  {p}")

print("\n=== hl-mainnet-node-data ===")
for p in list_prefixes('hl-mainnet-node-data'):
    print(f"  {p}")

=== hyperliquid-archive ===
  Testnet/
  asset_ctxs/
  market_data/

=== hl-mainnet-node-data ===
  explorer_blocks/
  misc_events_by_block/
  node_fills/
  node_fills_by_block/
  node_trades/
  replica_cmds/


---

## Bucket 1: `hyperliquid-archive`

Market data archives. Not the focus for trader analysis, but useful for orderbook studies.

| Dataset | Date Range | Content | Format |
|---------|------------|---------|--------|
| `market_data` | Apr 2023 - Present | L2 orderbook snapshots per coin/hour | JSON+LZ4 |
| `asset_ctxs` | May 2023 - Present | Daily asset context (funding, OI, etc.) | CSV+LZ4 |

In [20]:
# Discover date ranges for hyperliquid-archive
def fmt_date(d):
    return datetime.strptime(d, "%Y%m%d").strftime("%b %d, %Y")


print("=" * 70)
print(f"{'Dataset':<20} {'First':<15} {'Last':<15} {'Days':>6}")
print("=" * 70)

# market_data (organized by date folders)
dates = list_prefixes("hyperliquid-archive", "market_data/")
if dates:
    first = dates[0].split("/")[-2]
    last = dates[-1].split("/")[-2]
    print(f"{'market_data':<20} {fmt_date(first):<15} {fmt_date(last):<15} {len(dates):>6}")

# asset_ctxs (files named by date)
files = list_files("hyperliquid-archive", "asset_ctxs/", limit=1000)
if files:
    # list_files returns (key, size) tuples - extract keys
    file_dates = [key.split("/")[-1].split(".")[0] for key, size in files]
    first = file_dates[0]
    last = file_dates[-1]
    print(f"{'asset_ctxs':<20} {fmt_date(first):<15} {fmt_date(last):<15} {len(files):>6}")

print("=" * 70)

Dataset              First           Last              Days
market_data          Apr 15, 2023    Nov 02, 2025       925
asset_ctxs           May 20, 2023    Nov 02, 2025       898


---

## Bucket 2: `hl-mainnet-node-data`

Node-streamed data. This is what we need for trader analysis.

| Dataset | Date Range | Content | Format |
|---------|------------|---------|--------|
| `node_fills_by_block` | **Jul 2025 - Present** | Fills with PnL, fees, maker/taker | JSON+LZ4 |
| `node_fills` | May 2025 - Jul 2025 | Fills (legacy format) | JSON+LZ4 |
| `node_trades` | Mar 2025 - Jun 2025 | Trades with buyer/seller | JSON+LZ4 |
| `replica_cmds` | Jul 2025 - Present | L1 transactions | JSON+LZ4 |
| `misc_events_by_block` | Jul 2025 - Present | Liquidations, funding, etc. | JSON+LZ4 |
| `explorer_blocks` | Feb 2023 - Present | Raw blocks (orders, cancels) | MessagePack+LZ4 |

**Best dataset**: `node_fills_by_block` — complete fill data, no reconstruction needed.

In [21]:
# Discover all date ranges from S3
def get_date_range(prefix):
    """Get first/last date for an hourly prefix"""
    dates = list_prefixes("hl-mainnet-node-data", f"{prefix}hourly/")
    if not dates:
        return None, None, 0
    first = dates[0].rstrip("/").split("/")[-1]
    last = dates[-1].rstrip("/").split("/")[-1]
    return first, last, len(dates)


def fmt_date(d):
    return datetime.strptime(d, "%Y%m%d").strftime("%b %d, %Y")


datasets = [
    ("node_fills_by_block/", "Fills by block (BEST)"),
    ("node_fills/", "Node fills (legacy)"),
    ("node_trades/", "Node trades (legacy)"),
    ("misc_events_by_block/", "Misc events"),
]

print("=" * 70)
print(f"{'Dataset':<25} {'First':<15} {'Last':<15} {'Days':>6}")
print("=" * 70)

for prefix, label in datasets:
    first, last, count = get_date_range(prefix)
    if first:
        print(f"{label:<25} {fmt_date(first):<15} {fmt_date(last):<15} {count:>6}")
    else:
        print(f"{label:<25} {'N/A':<15} {'N/A':<15} {0:>6}")

# replica_cmds - different structure (ISO timestamp folders, not hourly/)
replica_prefixes = list_prefixes("hl-mainnet-node-data", "replica_cmds/")
if replica_prefixes:
    first_ts = replica_prefixes[0].split("/")[1][:10]  # Extract date part
    last_ts = replica_prefixes[-1].split("/")[1][:10]
    print(f"{'Replica commands':<25} {first_ts:<15} {last_ts:<15} {len(replica_prefixes):>6}")

# Explorer blocks - different structure (by block number, not date)
explorer_prefixes = list_prefixes("hl-mainnet-node-data", "explorer_blocks/")
if explorer_prefixes:
    last_range = explorer_prefixes[-1].split("/")[1]
    print(f"{'Explorer blocks':<25} {'Block 0':<15} {f'Block {last_range}+':<15} {'N/A':>6}")

print("=" * 70)

Dataset                   First           Last              Days
Fills by block (BEST)     Jul 27, 2025    Nov 30, 2025       127
Node fills (legacy)       May 25, 2025    Jul 27, 2025        64
Node trades (legacy)      Mar 22, 2025    Jun 21, 2025        66
Misc events               Sep 27, 2025    Nov 30, 2025        65
Replica commands          2025-01-12      2025-11-29          52
Explorer blocks           Block 0         Block 800000000+    N/A


---

## Exploring `node_fills_by_block` (The Best Data)

Every fill since July 2025 with complete information.

In [22]:
# Find a date with data
dates = list_prefixes('hl-mainnet-node-data', 'node_fills_by_block/hourly/')
sample_date = dates[-2].rstrip('/').split('/')[-1]  # Second to last (likely complete)
print(f"Using date: {sample_date}")

# List files for this date (files are named {hour}.lz4, not in hour subfolders)
files = list_files('hl-mainnet-node-data', f'node_fills_by_block/hourly/{sample_date}/')
print(f"Files for this date: {len(files)}")
print("Sample files:", [k.split('/')[-1] for k, v in files[:5]])

Using date: 20251129
Files for this date: 24
Sample files: ['0.lz4', '1.lz4', '10.lz4', '11.lz4', '12.lz4']


In [23]:
# Download and parse hour 12 (midday, typically active)
sample_key = f'node_fills_by_block/hourly/{sample_date}/12.lz4'
data = download('hl-mainnet-node-data', sample_key)

# Each line is a block with multiple fill events
blocks = list(parse_jsonl_lz4(data))
print(f"Loaded {len(blocks):,} blocks from hour 12")

# Flatten to individual fills: each event is [user_address, fill_data]
fills = []
for block in blocks:
    for user, fill_data in block.get('events', []):
        fill_data['user'] = user
        fill_data['block_time'] = block['block_time']
        fills.append(fill_data)

print(f"Extracted {len(fills):,} fills\n")
print("Sample fill:")
print(json.dumps(fills[0], indent=2))

Loaded 44,641 blocks from hour 12
Extracted 170,978 fills

Sample fill:
{
  "coin": "WLFI",
  "px": "0.15995",
  "sz": "77.0",
  "side": "B",
  "time": 1764417600000,
  "startPosition": "-22206.0",
  "dir": "Close Short",
  "closedPnl": "0.00462",
  "hash": "0x0000000000000000000000000000000000000000000000000000000000000000",
  "oid": 252520932522,
  "crossed": false,
  "fee": "0.0",
  "tid": 95174656478359,
  "cloid": "0x20251129000000000000000000187575",
  "feeToken": "USDC",
  "twapId": null,
  "user": "0x4264b5a132e4f263d6de2e0d01512a99ea21ec6e",
  "block_time": "2025-11-29T12:00:00.000242049"
}


### Data Structure

Each file contains **blocks** (one per line), each block contains multiple **fill events**:

```
{block_time, block_number, events: [[user_address, fill_data], ...]}
```

### Fill Schema

| Field | Description |
|-------|-------------|
| `user` | Wallet address (from event tuple) |
| `coin` | Asset traded |
| `px`, `sz` | Price and size |
| `side` | B (buy) or A (ask/sell) |
| `dir` | Direction: Open Long, Open Short, Close Long, Close Short, Long > Short, Short > Long |
| `closedPnl` | Realized PnL (only on closes) |
| `fee` | Fee paid (negative = rebate) |
| `crossed` | true = taker, false = maker |
| `startPosition` | Position before this fill |

In [24]:
# Quick stats
print(f"Unique users: {len(set(f['user'] for f in fills))}")
print(f"Unique coins: {len(set(f['coin'] for f in fills))}")
print(f"\nDirections:")
for d, c in Counter(f['dir'] for f in fills).most_common():
    print(f"  {d}: {c}")

Unique users: 5767
Unique coins: 239

Directions:
  Open Long: 39744
  Close Long: 37126
  Open Short: 36759
  Close Short: 34161
  Buy: 10371
  Sell: 10371
  Long > Short: 1233
  Short > Long: 1213


---

## Exploring `explorer_blocks` (Raw Historical Data)

For Feb 2023 - Jul 2025, only raw blocks exist. Must reconstruct fills.

In [25]:
# Get earliest block file
block_files = list_files("hl-mainnet-node-data", "explorer_blocks/0/0/", limit=5)
print("Early block files:")
for key, size in block_files:
    print(f"  {key}")

Early block files:
  explorer_blocks/0/0/1000.rmp.lz4
  explorer_blocks/0/0/10000.rmp.lz4
  explorer_blocks/0/0/100000.rmp.lz4
  explorer_blocks/0/0/10100.rmp.lz4
  explorer_blocks/0/0/10200.rmp.lz4


In [26]:
# Download and parse
if block_files:
    key, size = block_files[0]
    data = download("hl-mainnet-node-data", key)
    blocks = parse_msgpack_lz4(data)
    print(f"Loaded {len(blocks)} blocks")
    print(f"Block range: {blocks[0]['header']['height']} - {blocks[-1]['header']['height']}")
    print(f"Time: {blocks[0]['header']['block_time']}")

Loaded 100 blocks
Block range: 901 - 1000
Time: 2023-02-26T17:41:39.942659


In [27]:
# Action types in blocks
if block_files:
    actions = Counter()
    for block in blocks:
        for tx in block.get('txs', []):
            for action in tx.get('actions', []):
                actions[action.get('type', 'unknown')] += 1
    
    print("Action types:")
    for t, c in actions.most_common():
        print(f"  {t}: {c}")

Action types:
  order: 100
  cancel: 25
  SetGlobalAction: 5
  CreditBridgeDepositAction: 1
  connect: 1


### What's in Raw Blocks

- Orders (asset, side, price, size)
- Cancels
- User addresses

### What's Missing

- Fills
- PnL
- Fees

---

## Our Approach

**Use `node_fills_by_block`** (Jul 2025 - Present)

- ~5 months of complete fill data
- Every trader, every fill, with PnL and fees
- Emphasize trader analysis
- No reconstruction needed


## Reconstruction: Theoretical

To get pre-July 2025 fills, we can reconstruct because matching is deterministic. Same orders + same sequence = same fills.

**Challenges:**
- Block explorer data is huge
- Must track order book state
- Edge cases: liquidations, funding, trigger orders (would use misc_events_by_block)
- Would need to rewrite the entire matching engine (their node is closed source docker binary)

**Alternative:**
- Can get two months of fill data from `node_fills` (legacy format)
- Best effort and fetch latest 10k trades. Reconcile with other data providers that may have more.